In [23]:
import sys
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.model_selection import train_test_split
import pandas as pd
import qgrid

sys.path.append('..')
from urlclustering.noise_word import NoiseWordDetector, transform_feature
from urlclustering.toolkit.noise_sample import read_sample, sample_columns, sample_feature_columes

pd.set_option('display.max_rows', 1000)

In [29]:
samples = read_sample('../data/samples_meregs.csv')
X, y = transform_feature(samples)

In [30]:
samples_test = read_sample('../data/samples.csv')
X_test, y_test = transform_feature(samples_test)

In [6]:
samples_all = pd.concat([samples, samples_test])
X_train, y_train = transform_feature(samples_all)

In [31]:
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X, y)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25)
#samples_test = pd.DataFrame(X_test, columns=NoiseWordDetector.columns)

In [32]:
detector = NoiseWordDetector()
detector.fit(X_train, y_train)

In [33]:
y_score = detector.predict(X_test)
average_precision = average_precision_score(y_test, y_score)
precision, recall, threshold = precision_recall_curve(y_test, y_score)

In [34]:
samples_test['y_score'] = y_score

In [35]:
print(f'{"threshold":<12} {recall}')
print(f'{"precision":<12} {precision}')
print(f'{"recall":<12} {recall}')

threshold    [1. 0.]
precision    [0.99999716 1.        ]
recall       [1. 0.]


In [39]:
qgrid.show_grid(samples_test.loc[(samples_test.label == False) & (samples_test.y_score == False), ['word', 'url'] + sample_feature_columes + ['label','y_score']])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [47]:
qgrid.show_grid(samples_test.loc[samples_test.label != samples_test.y_score, ['word'] + sample_feature_columes + ['label','y_score']])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [ ]:
samples_test[samples_test.label != samples_test.y_score]